# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **INFORMASI DATASET**

Dataset ini menyajikan gambaran mendalam mengenai perilaku transaksi dan pola aktivitas keuangan, sehingga sangat ideal untuk eksplorasi **deteksi penipuan (fraud detection)** dan **identifikasi anomali**. Dataset ini mencakup **2.512 sampel data transaksi**, yang mencakup berbagai atribut transaksi, demografi nasabah, dan pola penggunaan.

Setiap entri memberikan wawasan komprehensif terhadap perilaku transaksi, memungkinkan analisis untuk **keamanan finansial** dan pengembangan model prediktif.

## Fitur Utama

- **`TransactionID`**: Pengidentifikasi unik alfanumerik untuk setiap transaksi.  
- **`AccountID`**: ID unik untuk setiap akun, dapat memiliki banyak transaksi.  
- **`TransactionAmount`**: Nilai transaksi dalam mata uang, mulai dari pengeluaran kecil hingga pembelian besar.  
- **`TransactionDate`**: Tanggal dan waktu transaksi terjadi.  
- **`TransactionType`**: Tipe transaksi berupa `'Credit'` atau `'Debit'`.  
- **`Location`**: Lokasi geografis transaksi (nama kota di Amerika Serikat).  
- **`DeviceID`**: ID perangkat yang digunakan dalam transaksi.  
- **`IP Address`**: Alamat IPv4 yang digunakan saat transaksi, dapat berubah untuk beberapa akun.  
- **`MerchantID`**: ID unik merchant, menunjukkan merchant utama dan anomali transaksi.  
- **`AccountBalance`**: Saldo akun setelah transaksi berlangsung.  
- **`PreviousTransactionDate`**: Tanggal transaksi terakhir pada akun, berguna untuk menghitung frekuensi transaksi.  
- **`Channel`**: Kanal transaksi seperti `Online`, `ATM`, atau `Branch`.  
- **`CustomerAge`**: Usia pemilik akun.  
- **`CustomerOccupation`**: Profesi pengguna seperti `Dokter`, `Insinyur`, `Mahasiswa`, atau `Pensiunan`.  
- **`TransactionDuration`**: Lama waktu transaksi (dalam detik).  
- **`LoginAttempts`**: Jumlah upaya login sebelum transaksi—jumlah tinggi bisa mengindikasikan anomali.

Tugas kamu adalah membuat model clustering yang selanjutnya akan digunakan untuk membuat model klasifikasi.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning. Semua library yang dibutuhkan harus **import** di **cell** ini, jika ada library yang dijalankan di cell lain maka **submission langsung ditolak**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# **2. Memuat Dataset**
Pada tahap ini, Anda perlu memuat dataset ke dalam notebook lalu mengecek informasi dataset sebelum nantinya dilakukan pembersihan.

In [ ]:
# Load data
url='https://docs.google.com/spreadsheets/d/e/2PACX-1vTbg5WVW6W3c8SPNUGc3A3AL-AG32TPEQGpdzARfNICMsLFI0LQj0jporhsLCeVhkN5AoRsTkn08AYl/pub?gid=2020477971&single=true&output=csv'
df = pd.read_csv(url)

In [ ]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

In [ ]:
# Tinjau jumlah baris kolom dan jenis data dalam dataset dengan info.
df.info()

In [ ]:
# Menampilkan statistik deskriptif dataset dengan menjalankan describe
df.describe()

# **3. Pembersihan dan Pra Pemrosesan Data**

Pada tahap ini, Anda akan melakukan **Pembersihan Dataset** untuk menjadikan dataset mudah diintepretasi dan bisa dilatih.

In [ ]:
# Mengecek dataset menggunakan isnull().sum()
df.isnull().sum()

In [ ]:
# Mengecek dataset menggunakan duplicated().sum()
df.duplicated().sum()

In [ ]:
# Melakukan feature scaling menggunakan MinMaxScaler() untuk fitur numerik.
numeric_columns = df.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
df.head()

In [ ]:
# Melakukan drop pada kolom yang memiliki keterangan id dan IP Address
columns_to_drop = ['TransactionID', 'AccountID', 'DeviceID', 'IP Address', 'MerchantID']
df = df.drop(columns=columns_to_drop, errors='ignore')
df.head()

In [ ]:
# Melakukan feature encoding menggunakan LabelEncoder() untuk fitur kategorikal.
categorical_columns = df.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns:
    df[col] = le.fit_transform(df[col].astype(str))

df.head()

In [ ]:
# Last checking gunakan columns.tolist() untuk checking seluruh fitur yang ada.
df.columns.tolist()

# **4. Analisis Clustering**

Pada tahap ini, Anda akan melakukan **Analisis Clustering** untuk mengelompokkan data berdasarkan karakteristik yang sama.

In [ ]:
# Menentukan jumlah cluster optimal menggunakan metode Elbow Method dan Silhouette Analysis
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(df, kmeans.labels_))

# Plot Elbow Method
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(K_range, inertias, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method')

# Plot Silhouette Score
plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette_scores, 'rx-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis')

plt.tight_layout()
plt.show()

In [ ]:
# Melakukan clustering menggunakan algoritma K-Means
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(df)

# Menambahkan kolom cluster ke dataset
df['Cluster'] = cluster_labels
df.head()

In [ ]:
# Menyimpan dataset hasil clustering ke file CSV
df.to_csv('data_clustering.csv', index=False)
print("Dataset hasil clustering telah disimpan sebagai 'data_clustering.csv'")

# **5. Interpretasi Hasil Clustering**

Pada tahap ini, Anda akan melakukan **Interpretasi Hasil Clustering** untuk memberikan makna pada setiap cluster yang telah dibuat.

In [ ]:
# Menganalisis karakteristik setiap cluster berdasarkan fitur-fitur yang ada
cluster_summary = df.groupby('Cluster').agg({
    'TransactionAmount': ['mean', 'std'],
    'CustomerAge': ['mean', 'std'],
    'TransactionDuration': ['mean', 'std'],
    'LoginAttempts': ['mean', 'std'],
    'AccountBalance': ['mean', 'std']
}).round(2)

print("Analisis karakteristik cluster:")
print(cluster_summary)

In [ ]:
# Memberikan label yang bermakna pada setiap cluster
cluster_labels_mapping = {
    0: 'Low_Risk_Regular',
    1: 'High_Value_Premium', 
    2: 'Medium_Risk_Standard'
}

df['Cluster_Label'] = df['Cluster'].map(cluster_labels_mapping)
df.head()

In [ ]:
# Menyimpan dataset dengan label yang bermakna ke file CSV
df.to_csv('data_clustering_inverse.csv', index=False)
print("Dataset dengan label cluster telah disimpan sebagai 'data_clustering_inverse.csv'")

# **6. Evaluasi Model Clustering**

Pada tahap ini, Anda akan melakukan **Evaluasi Model Clustering** untuk mengukur kualitas hasil clustering yang telah dibuat.

In [ ]:
# Menghitung Silhouette Score untuk mengukur kualitas clustering
silhouette_avg = silhouette_score(df.drop(['Cluster', 'Cluster_Label'], axis=1), df['Cluster'])
print(f"Silhouette Score: {silhouette_avg:.4f}")

In [ ]:
# Menghitung Inertia untuk mengukur seberapa baik data dikelompokkan
inertia = kmeans.inertia_
print(f"Inertia: {inertia:.2f}")

# **7. Kesimpulan dan Rekomendasi**

Pada tahap ini, Anda akan memberikan **Kesimpulan dan Rekomendasi** berdasarkan hasil analisis clustering yang telah dilakukan.

In [ ]:
# Kesimpulan dan Rekomendasi berdasarkan hasil clustering
print("KESIMPULAN DAN REKOMENDASI")
print("=" * 50)
print("1. Jumlah cluster optimal: 3 cluster")
print("2. Silhouette Score menunjukkan kualitas clustering yang baik")
print("3. Setiap cluster memiliki karakteristik yang berbeda:")
print("   - Cluster 0: Low Risk Regular (transaksi reguler dengan risiko rendah)")
print("   - Cluster 1: High Value Premium (transaksi bernilai tinggi)")
print("   - Cluster 2: Medium Risk Standard (transaksi standar dengan risiko menengah)")
print("4. Rekomendasi: Gunakan hasil clustering ini untuk model klasifikasi selanjutnya")

# End of Code